In [1]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup


### To get coordinates of Lucknow

In [13]:
address = 'New Delhi, India'

geolocator = Nominatim(user_agent = 'ny_users')
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude
print('The Geograpical Co-ordinate of Neighborhood_1 are {}, {}.'.format(latitude_n1, longitude_n1))

The Geograpical Co-ordinate of Neighborhood_1 are 28.6138954, 77.2090057.


In [14]:
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # my Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET: 1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


### To get json 

In [15]:
radius = 10000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

### To normalize and get all columns

In [16]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.postalCode', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups',
       'venue.location.neighborhood', 'venue.venuePage.id'],
      dtype='object')

### To filter columns

In [17]:
filtered_columns = ['venue.name', 'venue.categories','venue.location.address', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng
0,Tamra,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...","19, Ashoka Rd",28.620543,77.218174
1,The Imperial,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",Janpath,28.625548,77.218664
2,Pandey Paan,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...","Shop No 3, North Avenue",28.622249,77.201075
3,Varq | वर्क,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...","Taj Mahal Hotel, Lower Lobby Level",28.604547,77.223781
4,Naturals Ice Cream,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",L Block Outer Circle,28.634455,77.222139


### To get category

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(50)

,name,categories,address,lat,lng
0,Tamra,Restaurant,"19, Ashoka Rd",28.620543,77.218174
1,The Imperial,Hotel,Janpath,28.625548,77.218664
2,Pandey Paan,Smoke Shop,"Shop No 3, North Avenue",28.622249,77.201075
3,Varq | वर्क,Indian Restaurant,"Taj Mahal Hotel, Lower Lobby Level",28.604547,77.223781
4,Naturals Ice Cream,Ice Cream Shop,L Block Outer Circle,28.634455,77.222139
5,The Big Chill Cafe,Italian Restaurant,68A,28.600686,77.227636
6,SODABOTTLEOPENERWALA,Irani Cafe,Khan Market,28.600141,77.226273
7,HOTEL SARAVANA BHAVAN,South Indian Restaurant,"46, Janpath",28.627041,77.219514
8,Connaught Place | कनॉट प्लेस (Connaught Place),Plaza,Connaught Place,28.632731,77.220018
9,Gulati Restaurant,Indian Restaurant,Pandara Road,28.608010,77.229989


In [20]:
map = folium.Map(location =[latitude_n1,longitude_n1] , zoom_start = 13 )
for lat , lng , label in zip(nearby_venues.lat , nearby_venues.lng , nearby_venues.name):
    folium.CircleMarker([lat,lng] , 
                       radius = 5,
                       popuop = label , 
                       fill = True ,
                       color = 'blue' ,
                       fill_color = 'black' ,
                       fill_opacity = 0.5).add_to(map)

In [21]:
map

In [22]:
c= pd.Series(nearby_venues.categories)
c.value_counts()

Indian Restaurant                  18
Café                               10
Hotel                              10
Italian Restaurant                  5
Lounge                              4
South Indian Restaurant             4
Bakery                              3
Restaurant                          3
Hotel Bar                           2
Mediterranean Restaurant            2
Bar                                 2
Art Gallery                         2
French Restaurant                   2
Arcade                              2
Tibetan Restaurant                  2
Food                                1
Comfort Food Restaurant             1
University                          1
Asian Restaurant                    1
Golf Course                         1
Clothing Store                      1
Vegetarian / Vegan Restaurant       1
Chinese Restaurant                  1
Molecular Gastronomy Restaurant     1
Fast Food Restaurant                1
English Restaurant                  1
Plaza       